In [1]:
"""
Dependencies for loudness map visualization
"""
import os
import numpy as np
import json
import matplotlib.pyplot as plt
from librosa.feature import rms
from scipy.spatial.transform import Rotation as T

# NeRAF Loudness Maps Visualization

This notebook illustrates how to generate and visualize acoustic loudness maps using NeRAF. The workflow includes:
1. Creating a dense grid of microphone poses within a scene
2. Rendering RIRs using NeRAF
3. Computing loudness values across the grid
4. Visualizing the loudness distribution as a heatmap

**Prerequisites:** A trained NeRAF model and nerfstudio installation

In [ ]:
# Configuration - Update these paths to match your setup
path_data = './NeRAF_loudness_maps'  # Output directory for rendered audio
path_dataset = '/path/to/raf_dataset'  # Dataset root
room = os.path.join(path_dataset, "FurnishedRoom")  # Specific room dataset

## Step 1: Create Dense Grid of Microphone Poses

This section creates a dense grid of microphone poses at multiple heights and a fixed source pose, then prepares this data for NeRAF rendering.

In [ ]:
def _process_poses(files, room):
    """
    Process microphone and source positions from dataset files.
    
    Args:
        files: List of file identifiers to process
        room: Root directory containing the room data
        
    Returns:
        Dictionary with keys: 'rot', 'mic_pose', 'source_pose', 'rot_degree'
    """
    poses_list = []
    
    for f in files:
        rx_file = os.path.join(room, 'data', f, 'rx_pos.txt')  # Receiver (microphone) file
        tx_file = os.path.join(room, 'data', f, 'tx_pos.txt')  # Transmitter (source) file
        
        # Load receiver positions
        with open(rx_file, 'r') as file:
            rx = file.readlines()
            rx = [line.replace('\n', '').split(',') for line in rx]
            rx = [[float(j) for j in i] for i in rx]
            rx_pose = np.array(rx[0])
        
        # Load transmitter: quaternion (xyzW) and position
        with open(tx_file, 'r') as file:
            tx = file.readlines()
            tx = [line.replace('\n', '').split(',') for line in tx]
            tx = [[float(j) for j in i] for i in tx]
            tx = np.array(tx[0])
        
        # Extract quaternion and pose
        quat = tx[:4]  # Quaternion format: xyzW
        tx_pose = tx[4:]  # Source position
        
        # Convert quaternion to rotation using Euler angles (YXZ convention)
        r = T.from_quat(quat)
        spk_rot_euler = r.as_euler('yxz', degrees=True)
        spk_rot_deg = np.round(spk_rot_euler[0], decimals=0)  # Y-axis rotation
        
        # Convert rotation degree to a normalized vector for SHE (Spherical Harmonics)
        rad_spk_rot = np.deg2rad(spk_rot_deg)
        spk_rot_vec = np.array([np.cos(rad_spk_rot), 0, np.sin(rad_spk_rot)])
        spk_rot_norm = (spk_rot_vec + 1.0) / 2.0  # Normalize to [0, 1]
        
        # Store pose information
        poses_list.append({
            'mic_pose': np.expand_dims(rx_pose, axis=0),
            'source_pose': np.expand_dims(tx_pose, axis=0),
            'rot': np.expand_dims(spk_rot_norm, axis=0),
            'rot_degree': np.expand_dims(np.array([spk_rot_deg], dtype=float), axis=0)
        })
    
    # Concatenate all poses
    result = {
        'mic_pose': np.concatenate([p['mic_pose'] for p in poses_list], axis=0),
        'source_pose': np.concatenate([p['source_pose'] for p in poses_list], axis=0),
        'rot': np.concatenate([p['rot'] for p in poses_list], axis=0),
        'rot_degree': np.concatenate([p['rot_degree'] for p in poses_list], axis=0)
    }
    
    return result

In [ ]:
# Load dataset split information (which scenes are in train/test)
split_file = os.path.join(room, 'metadata/data-split.json')
with open(split_file) as f:
    split_data = json.load(f)

split_files_train = split_data['train'][0]
split_files_test = split_data['test'][0]
split_files_all = split_files_train + split_files_test

print(f"Train files: {len(split_files_train)}, Test files: {len(split_files_test)}")

In [ ]:
## Process all poses and compute scene boundaries
poses = _process_poses(split_files_all, room)

# Calculate AABB (Axis-Aligned Bounding Box) of scene
aabb = np.array([poses['mic_pose'].min(axis=0), poses['mic_pose'].max(axis=0)])
print(f"Scene AABB (min): {aabb[0]}")
print(f"Scene AABB (max): {aabb[1]}")

In [ ]:
# Extract microphone and source poses 
p_mic = poses["mic_pose"]  # Shape: (N, 3) - all microphone positions
p_source = poses["source_pose"]  # Shape: (N, 3) - all source positions

In [ ]:
# Visualize top-down view of scene with all microphone (blue) and source (orange) positions
# This helps in selecting an appropriate source location for rendering
plt.figure(figsize=(8, 8))
plt.scatter(p_mic[:, 2], p_mic[:, 0], s=1, alpha=0.5, label='Microphones')
plt.scatter(p_source[:, 2], p_source[:, 0], s=20, alpha=0.7, label='Sources')
plt.xlabel('Z (m)')
plt.ylabel('X (m)')
plt.title('Scene Layout (Top-Down View)')
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# Create a dense regular grid of microphone poses for rendering
step = 0.1  # Density

# Add margin around the AABB to ensure full scene coverage
# Note: Positions outside the scene will be ignored in later visualization
margin = 0.5
min_x = aabb[0][0] - margin
max_x = aabb[1][0] + margin
min_y = aabb[0][2] - margin
max_y = aabb[1][2] + margin

# Extract unique heights from the original microphone positions
microphone_heights = np.unique(np.round(p_mic[:, 1], decimals=1))

# Create 3D grid using meshgrid
x, y, z = np.meshgrid(
    np.arange(min_x, max_x, step),  # X positions
    np.arange(min_y, max_y, step),  # Y positions (horizontal distance)
    microphone_heights  # Z positions (heights)
)

# Flatten to create list of poses
x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()

print(f"Generated {len(x_flat)} microphone poses")

In [ ]:
# Select a source position and rotation for rendering
idx_source = 0  # Which source to use
source_pose_selected = p_source[idx_source]

# Select source rotation (azimuth angle in degrees)
# This controls the direction the speaker is "facing"
rotation_deg = 65

# Convert rotation to normalized rotation vector for NeRAF
rad_rotation = np.deg2rad(rotation_deg)
rot_vector = np.array([np.cos(rad_rotation), 0, np.sin(rad_rotation)])
source_rotation_normalized = (rot_vector + 1.0) / 2.0  # Normalize to [0, 1]

print(f"Source position: {source_pose_selected}")
print(f"Source rotation: {rotation_deg}°")

In [ ]:
# Visualize the dense grid overlaid on the original scene
plt.figure(figsize=(10, 10), dpi=500)
plt.scatter(p_mic[:, 2], p_mic[:, 0], s=0.5)
plt.scatter(p_source[:, 2], p_source[:, 0], s=0.5)
plt.plot(y_flat, x_flat, 'x')
plt.scatter(source_pose_selected[2], source_pose_selected[0], c='r')
plt.gca().set_aspect('equal', adjustable='box')
#plt.savefig('layout.png',dpi=500, transparent=True)
plt.show()

In [ ]:
# Format poses for NeRAF rendering
# NeRAF expects poses in (X, Z, Y) order
pos_grid_for_neraf = np.array([x_flat, z_flat, y_flat]).T

# Create dictionary in NeRAF format
poses_for_rendering = {
    'rots': source_rotation_normalized,  # Source rotation
    'mic_poses': pos_grid_for_neraf,     # Microphone positions
    'source_poses': source_pose_selected  # Source position
}

In [ ]:
# Save poses for NeRAF rendering
output_filename = f"loudness_S{idx_source}_Rot{rotation_deg}.npy"
output_path = os.path.join(path_data, output_filename)
np.save(output_path, poses_for_rendering)
print(f"Saved rendering poses to: {output_path}")

## Step 2: Render and Compute Loudness Maps

### Rendering with NeRAF

Before running the following cells, you must generate Room Impulse Responses (RIRs) using NeRAF:

```bash
AVN_RENDER_POSES=your_poses.npy ns-eval \
  --load-config path/to/config.yml \
  --render-output-path ./output_folder
```

**Parameters:**
- `AVN_RENDER_POSES`: Path to the .npy file with poses (created in Step 1)
- `config.yml`: NeRAF model configuration file
- `output_folder`: Where rendered audio files will be saved

The command will generate RIR audio STFT (in .npy format) for each microphone pose.

In [ ]:
# Load rendered audio and corresponding poses
# Update these paths to match your rendering output
audio_data_dir = os.path.join(path_data, f'loudness_S{idx_source}_Rot{rotation_deg}')  # Directory with rendered RIRs
poses_file = os.path.join(path_data, f'loudness_S{idx_source}_Rot{rotation_deg}.npy')  # Poses file from Step 1

poses_loc = np.load(poses_file, allow_pickle=True).item()
print(f"Loaded poses from: {poses_file}")
print(f"Audio directory: {audio_data_dir}")

In [ ]:
# Extract and prepare poses for analysis
mic_poses = poses_loc['mic_poses']  # Shape: (N, 3)
source_poses = poses_loc['source_poses']  # Single source position
rots = poses_loc['rots']  # Single source rotation

print(f"Loaded {mic_poses.shape[0]} microphone poses")
print(f"Source position shape: {source_poses.shape}")

# Replicate source position and rotation for each microphone pose
# (one fixed source, multiple microphone positions)
rots_expanded = np.expand_dims(rots, axis=0)
rots_expanded = np.repeat(rots_expanded, mic_poses.shape[0], axis=0)

source_poses_expanded = np.expand_dims(source_poses, axis=0)
source_poses_expanded = np.repeat(source_poses_expanded, mic_poses.shape[0], axis=0)

In [ ]:
# Find and sort all rendered RIR audio files
audio_files = sorted([f for f in os.listdir(audio_data_dir) if f.endswith('.npy')])
print(f"Found {len(audio_files)} audio files")

In [ ]:
# Compute loudness (RMS) for each audio file
# This can be time-consuming depending on grid density
loudness_values = []

for i, audio_file in enumerate(audio_files):
    if (i + 1) % max(1, len(audio_files) // 10) == 0:
        print(f"Processing {i+1}/{len(audio_files)}...")
    
    audio_data = np.load(os.path.join(audio_data_dir, audio_file))
    # Clip to convert log-domain to linear domain
    audio_linear = np.clip(np.exp(audio_data) - 1e-3, 0.0, 10000.0)
    # Compute RMS (loudness) over the entire signal
    loudness = rms(S=audio_linear[0], frame_length=audio_linear[0].shape[-2] * 2 - 2)
    loudness_values.append(loudness)

loudness_values = np.array(loudness_values)
print(f"Computed loudness for {len(loudness_values)} positions")

In [ ]:
# Aggregate loudness values by (X, Z) position at different heights
# Filter to keep only microphones within a reasonable height range
height_min = 0.0  # Minimum height in meters
height_max = 2.0  # Maximum height in meters

position_dict = {}

for i in range(mic_poses.shape[0]):
    loudness_val = loudness_values[i]
    x_pos = mic_poses[i][0]
    z_pos = mic_poses[i][2]
    height = mic_poses[i][1]
    
    # Skip if outside height range
    if height < height_min or height > height_max:
        continue
    
    # Use (X, Z) as key for map
    key = (x_pos, z_pos)
    if key in position_dict:
        position_dict[key].append(loudness_val)
    else:
        position_dict[key] = [loudness_val]

print(f"Aggregated to {len(position_dict)} unique (X, Z) positions")

In [ ]:
# Extract coordinates and corresponding loudness values
x_coords = []
z_coords = []
loudness_lists = []

for (x, z) in sorted(position_dict.keys()):
    x_coords.append(x)
    z_coords.append(z)
    loudness_lists.append(position_dict[(x, z)])

x_coords = np.array(x_coords)
z_coords = np.array(z_coords)

In [ ]:
# Visualize loudness map as a heatmap
fig, ax = plt.subplots(figsize=(10, 10), dpi=500)

# Create scatter plot with color representing loudness
scatter = ax.scatter(z_coords, x_coords, c=np.log(np.sum(loudness_lists, axis=(1, 2, 3))), s=9, marker='s', 
                    cmap='coolwarm')

# Mark source position
ax.scatter(source_poses_expanded[0, 2], source_poses_expanded[0, 0], c='red')
ax.set_aspect('equal', 'box')

cbar = fig.colorbar(scatter, ax=ax)
# plt.savefig(f'path/to/loudness_S{idx_source}_Rot{rotation_deg}.png', dpi=500, transparent=True)

plt.show()

## Step 3: Create Final Visualizations

### Further Processing

To create a final publication-ready figure:

1. **Obtain scene geometry:** Get a top-down view/mesh from the dataset
2. **Overlay:** Save the loudness heatmap and overlay it on the scene geometry
3. **Crop:** Remove areas outside the scene bounds for a cleaner visualization
4. **Annotate:** Add source location marker (red dot) and other annotations as needed

To ease the superposition of the loudness map on the top view, refer back to the "Scene Layout" visualization from Step 1.